In [ ]:
import requests
import os
import urllib
import time
from bs4 import BeautifulSoup
parser = 'html.parser'

In [ ]:
from http.client import IncompleteRead


def download_video(filename, video_url, referer):
    #referer here is the page link
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "DNT": "1",
        "Connection": "keep-alive",
        "Referer": referer,
        "Cookie": "PHPSESSID=2m9oh764m7jj8up961akle7e7s; login_data=a%3A2%3A%7Bs%3A8%3A%22username%22%3Bs%3A6%3A%22kkksdf%22%3Bs%3A8%3A%22password%22%3Bs%3A32%3A%22bf13a2c7dca214b4c0b23feece14323a%22%3B%7D; zone-closed-4694230=true",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1"
    }

    filepath = (f"C:\\Users\\Kuljeet\\Desktop\\script\\videos\\{filename}.mp4")
    url = video_url.replace(" ", "%20")

    if (os.path.exists(filepath)):
        print("Found file hence skipped download")
        return

    print(f"Downloading file with filename: {filename}.mp4")
    while True:
        startTime = time.time()
        with urllib.request.urlopen(url) as response:
            body = None
            try:                                         
                body = response.read()  
            except IncompleteRead as e:
                body = e.partial
            except Exception as e:
                print(f"Exception occured while downloading video_url: {video_url}")
                continue
            with open(filepath, 'wb') as f:
                f.write(body)
            tot = str(time.time() - startTime)
            print(f"Downloaded file with filename: {filename} in {tot}ms")
            return

In [ ]:
def download_innerpage(page_url, inner_page_name, referer):
  cookies = {
    'PHPSESSID': 'kha6l6p9aa3fo1hq5a9r4krl9d',
  }

  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": referer,
    "DNT": "1",
    "Connection": "keep-alive",
    "Cookie": "PHPSESSID=2m9oh764m7jj8up961akle7e7s; login_data=a%3A2%3A%7Bs%3A8%3A%22username%22%3Bs%3A6%3A%22kkksdf%22%3Bs%3A8%3A%22password%22%3Bs%3A32%3A%22bf13a2c7dca214b4c0b23feece14323a%22%3B%7D; zone-closed-4694230=true",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin"
  }
  print(f"Downloading the page with url: {page_url}")
  if (os.path.exists(inner_page_name) is False):
    response = requests.request("GET", page_url, headers=headers, cookies=cookies)
    #save page
    with open(inner_page_name, 'w', encoding="utf-8") as f:
      f.write(response.text)

In [ ]:
for i in range(1, 30):
  main_page_name = (f"url")
  page_url = (f"url")
  
  print(f"Page no: {i} and page_url: {page_url}")
  print("######################################################################################")
  
  main_response = requests.get(page_url)
  with open(main_page_name, 'w') as f:
    f.write(main_response.text)

  with open(main_page_name, 'r') as main_file:
    content = main_file.read()
    soup = BeautifulSoup(content, parser)
    tags = soup.find_all('a', class_ = "thumb-img pop-execute")
    for tag in tags:
      page_link = tag['href']

      inner_page_name = page_link.split("/")[-1]
      download_innerpage(page_link, inner_page_name, page_url)

      print(inner_page_name)
      with open(inner_page_name, 'r', encoding="utf-8") as f:
        page_soup = BeautifulSoup(f.read(), parser)

        #div(detail-con) -> 2nd div among children -> 4th span -> 3rd atag
        atag = page_soup.find('a', title="Download")
        try:
          video_url = atag['href']
          print(f"Video url: {video_url}")

          query_params = {}
          param = video_url.split("?")[1]
          file1 = param.split("&")[0].split("=")[1]
          name2 = param.split("&")[1].split("=")[1]
          query_params["file"] = file1
          query_params["name"] = name2
          download_video(name2, video_url, page_link)
        except Exception as e:
          print(f"Exception occured with video url: {video_url} and page_link: {page_link}")